In [45]:
import torch
import torch.nn as nn
import albumentations as A
import numpy as np
import cv2
import os
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple, Dict, Optional
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchvision import models
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm
import logging
from torch.utils.data import random_split
from dataclasses import dataclass

In [28]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("anthonytherrien/dog-vs-cat")

print("Path to dataset files:", path)

Path to dataset files: /Users/daniel/.cache/kagglehub/datasets/anthonytherrien/dog-vs-cat/versions/3


In [29]:
path = Path(path)

for item in Path(path).iterdir():
    print(item)



/Users/daniel/.cache/kagglehub/datasets/anthonytherrien/dog-vs-cat/versions/3/animals


In [30]:
animal_dir = path / "animals"
animal_dir

PosixPath('/Users/daniel/.cache/kagglehub/datasets/anthonytherrien/dog-vs-cat/versions/3/animals')

In [40]:
animal_data = ImageFolder(animal_dir)
animal_data, animal_data.classes, animal_data.class_to_idx, animal_data.targets

(Dataset ImageFolder
     Number of datapoints: 1000
     Root location: /Users/daniel/.cache/kagglehub/datasets/anthonytherrien/dog-vs-cat/versions/3/animals,
 ['cat', 'dog'],
 {'cat': 0, 'dog': 1},
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [44]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

train_indices, val_indices = train_test_split(
    range(len(animal_data)), test_size=0.2, random_state=42, stratify=animal_data.targets
)
train_data = Subset(animal_data, train_indices)
test_data = Subset(animal_data, val_indices)
len(test_data), len(train_data), test_data, train_data

(200,
 800,
 <torch.utils.data.dataset.Subset at 0x3513d2870>)

In [47]:
@dataclass
class Config:
    batch_size: int = 32
    num_workers: int = 2
    pin_memory: bool = True
    aug_prob: float = 0.5
    transforms: Optional[A.Compose] = None
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    image_size: Tuple[int,int] = (224,224)
    learning_rate = 1e-3
    num_classes: int = 2
    num_epochs: int = 10

config = Config()